In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dirty_cafe_sales.csv')
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


### 1) Standardising column names

In [4]:
df.columns = df.columns.str.lower()

In [5]:
df.columns = df.columns.str.replace(" ", "_")

In [6]:
df.columns

Index(['transaction_id', 'item', 'quantity', 'price_per_unit', 'total_spent',
       'payment_method', 'location', 'transaction_date'],
      dtype='object')

### 2) Checking and removing any values that doesn't fit into respective column

On viewing the data earlier, it can be noticed some of the rows are filled with 'Unknown' and 'Error' keywords instead of leaving them empty.

So, it needs to be changed to np.nan data type

In [7]:
values_null = ["UNKNOWN", "ERROR"]
df.replace(values_null, np.nan, inplace= True)

### 3) Changing columns to respective data type

For instance, there are some numerical variables and date variables that need to be changed

In [8]:
def to_numeric(df, columns):
    for column in columns:
        df[column] = pd.to_numeric(df[column], errors= "coerce")
    return df.copy()


In [9]:
numeric = ['quantity', 'price_per_unit', 'total_spent']
df = to_numeric(df, columns= numeric)

In [10]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors = 'coerce')

### 4) Dealing with null values for numerical variables

Let us look at price variable with null values

In [11]:
df[df['price_per_unit'].isna()]

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
56,TXN_3578141,Cake,5.0,NaN,15.0,NaN,Takeaway,2023-06-27
65,TXN_4987129,Sandwich,3.0,NaN,NaN,NaN,In-store,2023-10-20
68,TXN_8427104,Salad,2.0,NaN,10.0,NaN,In-store,2023-10-27
85,TXN_8035512,Tea,3.0,NaN,4.5,Cash,NaN,2023-10-29
104,TXN_7447872,Juice,2.0,NaN,6.0,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...
9924,TXN_5981429,Juice,2.0,NaN,6.0,Digital Wallet,NaN,2023-12-24
9926,TXN_2464706,Cake,4.0,NaN,12.0,Digital Wallet,Takeaway,2023-11-09
9961,TXN_2153100,Tea,2.0,NaN,3.0,Cash,NaN,2023-12-29
9996,TXN_9659401,NaN,3.0,NaN,3.0,Digital Wallet,NaN,2023-06-02


On observing it can be found that price information is missing in some rows which can be filled by getting price for the respective item in non-null rows

So, we need to find a way to extract information so they can be mapped

In [12]:
items = [ x for x in df['item'].unique() if x is not np.nan]
items

['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'Sandwich', 'Juice', 'Tea']

In [13]:
df[df['item']=='Coffee']['price_per_unit'].value_counts().index

Index([2.0], dtype='float64', name='price_per_unit')

In [14]:
item_price_dictionary = {}

for item in items:
    item_price_dictionary[item] = df[df['item']==item]['price_per_unit'].value_counts().index

item_price_dictionary

{'Coffee': Index([2.0], dtype='float64', name='price_per_unit'),
 'Cake': Index([3.0], dtype='float64', name='price_per_unit'),
 'Cookie': Index([1.0], dtype='float64', name='price_per_unit'),
 'Salad': Index([5.0], dtype='float64', name='price_per_unit'),
 'Smoothie': Index([4.0], dtype='float64', name='price_per_unit'),
 'Sandwich': Index([4.0], dtype='float64', name='price_per_unit'),
 'Juice': Index([3.0], dtype='float64', name='price_per_unit'),
 'Tea': Index([1.5], dtype='float64', name='price_per_unit')}

In [15]:
for key in item_price_dictionary:
    item_price_dictionary[key] = item_price_dictionary[key][0]

item_price_dictionary

{'Coffee': np.float64(2.0),
 'Cake': np.float64(3.0),
 'Cookie': np.float64(1.0),
 'Salad': np.float64(5.0),
 'Smoothie': np.float64(4.0),
 'Sandwich': np.float64(4.0),
 'Juice': np.float64(3.0),
 'Tea': np.float64(1.5)}

In [16]:
df['price_per_unit'] = df['price_per_unit'].fillna(df['item'].map(item_price_dictionary))

In [17]:
df[df['price_per_unit'].isna()]

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
118,TXN_4633784,NaN,5.0,NaN,15.0,NaN,In-store,2023-02-06
151,TXN_4031509,NaN,4.0,NaN,16.0,Credit Card,Takeaway,2023-01-04
289,TXN_3495950,NaN,4.0,NaN,6.0,Credit Card,In-store,2023-02-19
334,TXN_2523298,NaN,4.0,NaN,6.0,NaN,In-store,2023-03-25
550,TXN_4186681,NaN,4.0,NaN,6.0,Digital Wallet,NaN,2023-05-24
750,TXN_5787508,NaN,3.0,NaN,9.0,Credit Card,Takeaway,2023-07-23
818,TXN_7940202,NaN,1.0,NaN,4.0,Digital Wallet,NaN,2023-07-23
1154,TXN_2473090,NaN,2.0,NaN,3.0,Credit Card,In-store,2023-03-03
1337,TXN_5031214,NaN,5.0,NaN,5.0,NaN,Takeaway,2023-07-29
1377,TXN_8396271,NaN,2.0,NaN,2.0,NaN,NaN,2023-09-12


Checking the null values of 'price_per_unit' after imputing with 'item' variable show there are still null values which shows clearly that there are no values in corresponding rows

However, we can fill the rows value in 'item' after computing 'price' information on the basis of 'quanity' and 'total_spent'  variables. This can be done by reversing the key, value pairs in the dictionary which will be done later

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    10000 non-null  object        
 1   item              9031 non-null   object        
 2   quantity          9521 non-null   float64       
 3   price_per_unit    9946 non-null   float64       
 4   total_spent       9498 non-null   float64       
 5   payment_method    6822 non-null   object        
 6   location          6039 non-null   object        
 7   transaction_date  9540 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 625.1+ KB


### 5) Computing the null values in quantity, price_per_unit and total_spent columns

I am trying to calculate missing values in respective columns by filtering the columns using mask_variable_name in order to avoid null value exception when performing division

### 1. Quantity

In [19]:
mask_quant = df['price_per_unit'].notna() & df['total_spent'].notna() 

In [20]:
df[mask_quant]['quantity'].isnull().sum()

np.int64(456)

In [21]:
df.loc[mask_quant, 'quantity'] = df.loc[mask_quant, 'total_spent']/df.loc[mask_quant, 'price_per_unit']

In [22]:
df[mask_quant]['quantity'].isnull().sum()

np.int64(0)

After filling the values, a check is performed to see wether all the values in the quantity column are filled or not. 

Above result clearly shows 456 empty rows before the operation are showing as zero after performing the respective row operations

In [23]:
df['quantity'].isnull().sum()

np.int64(23)

However, when the actual quantity column is checked without applying filter on dataframe it shows there are 23 null values

In [24]:
df[df['total_spent'].isnull()]

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
2,TXN_4271903,Cookie,4.0,1.0,NaN,Credit Card,In-store,2023-07-19
25,TXN_7958992,Smoothie,3.0,4.0,NaN,NaN,NaN,2023-12-13
31,TXN_8927252,NaN,2.0,1.0,NaN,Credit Card,NaN,2023-11-06
42,TXN_6650263,Tea,2.0,1.5,NaN,NaN,Takeaway,2023-01-10
65,TXN_4987129,Sandwich,3.0,4.0,NaN,NaN,In-store,2023-10-20
...,...,...,...,...,...,...,...,...
9893,TXN_3809533,Juice,2.0,3.0,NaN,Digital Wallet,Takeaway,2023-02-02
9954,TXN_1191659,Coffee,4.0,2.0,NaN,Credit Card,In-store,2023-11-21
9977,TXN_5548914,Juice,2.0,3.0,NaN,Digital Wallet,In-store,2023-11-04
9988,TXN_9594133,Cake,5.0,3.0,NaN,NaN,NaN,NaT


On verifying with the 'total spent' it shows that there are no value for respective rows of total spent column which makes it difficult to compute even if the price value is known

In [25]:
df[df['quantity'].isnull()]

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
236,TXN_8562645,Salad,NaN,5.0,NaN,NaN,In-store,2023-05-18
278,TXN_3229409,Juice,NaN,3.0,NaN,Cash,Takeaway,2023-04-15
641,TXN_2962976,Juice,NaN,3.0,NaN,NaN,NaN,2023-03-17
738,TXN_8696094,Sandwich,NaN,4.0,NaN,NaN,Takeaway,2023-05-14
2796,TXN_9188692,Cake,NaN,3.0,NaN,Credit Card,NaN,2023-12-01
3203,TXN_4565754,Smoothie,NaN,4.0,NaN,Digital Wallet,Takeaway,2023-10-06
3224,TXN_6297232,Coffee,NaN,2.0,NaN,NaN,NaN,2023-04-07
3401,TXN_3251829,Tea,NaN,1.5,NaN,Digital Wallet,In-store,2023-07-25
3779,TXN_7376255,NaN,NaN,NaN,25.0,NaN,In-store,2023-05-27
4257,TXN_6470865,Coffee,NaN,2.0,NaN,Digital Wallet,Takeaway,2023-09-18


After reviewing the data it makes total sense why some of quantity rows are still blank

### 2. Price

In [26]:
mask_price = df['quantity'].notna() & df['total_spent'].notna() 

In [27]:
df.loc[mask_price, 'price_per_unit'] = df.loc[mask_price, 'total_spent']/ df.loc[mask_price, 'quantity']

In [28]:
df[df['price_per_unit'].isna()]

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
1761,TXN_3611851,NaN,4.0,NaN,NaN,Credit Card,NaN,2023-02-09
2289,TXN_7524977,NaN,4.0,NaN,NaN,NaN,NaN,2023-12-09
3779,TXN_7376255,NaN,NaN,NaN,25.0,NaN,In-store,2023-05-27
4152,TXN_9646000,NaN,2.0,NaN,NaN,NaN,In-store,2023-12-14
7597,TXN_1082717,NaN,NaN,NaN,9.0,Digital Wallet,In-store,2023-12-13
9819,TXN_1208561,NaN,NaN,NaN,20.0,Credit Card,NaN,2023-08-19


### 3. Total spent

In [29]:
mask_spent = df['quantity'].notna() & df['price_per_unit'].notna()

In [30]:
df.loc[mask_spent, 'total_spent'] = df.loc[mask_spent, 'quantity'] * df.loc[mask_spent, 'price_per_unit']

In [31]:
df[df['total_spent'].isnull()]

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
236,TXN_8562645,Salad,NaN,5.0,NaN,NaN,In-store,2023-05-18
278,TXN_3229409,Juice,NaN,3.0,NaN,Cash,Takeaway,2023-04-15
641,TXN_2962976,Juice,NaN,3.0,NaN,NaN,NaN,2023-03-17
738,TXN_8696094,Sandwich,NaN,4.0,NaN,NaN,Takeaway,2023-05-14
1761,TXN_3611851,NaN,4.0,NaN,NaN,Credit Card,NaN,2023-02-09
2289,TXN_7524977,NaN,4.0,NaN,NaN,NaN,NaN,2023-12-09
2796,TXN_9188692,Cake,NaN,3.0,NaN,Credit Card,NaN,2023-12-01
3203,TXN_4565754,Smoothie,NaN,4.0,NaN,Digital Wallet,Takeaway,2023-10-06
3224,TXN_6297232,Coffee,NaN,2.0,NaN,NaN,NaN,2023-04-07
3401,TXN_3251829,Tea,NaN,1.5,NaN,Digital Wallet,In-store,2023-07-25


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    10000 non-null  object        
 1   item              9031 non-null   object        
 2   quantity          9977 non-null   float64       
 3   price_per_unit    9994 non-null   float64       
 4   total_spent       9977 non-null   float64       
 5   payment_method    6822 non-null   object        
 6   location          6039 non-null   object        
 7   transaction_date  9540 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 625.1+ KB


### 6) Impute item values based on 'price_per_unit'

 We can impute 'item' column based on 'price_per_unit' column by replacing key value pairs in item_price_dict

In [34]:
price_item_dict = {price:item for item, price in item_price_dictionary.items()}

price_item_dict

{np.float64(2.0): 'Coffee',
 np.float64(3.0): 'Juice',
 np.float64(1.0): 'Cookie',
 np.float64(5.0): 'Salad',
 np.float64(4.0): 'Sandwich',
 np.float64(1.5): 'Tea'}

In [35]:
df['item'] = df['item'].fillna(df['price_per_unit'].map(price_item_dict))

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    10000 non-null  object        
 1   item              9994 non-null   object        
 2   quantity          9977 non-null   float64       
 3   price_per_unit    9994 non-null   float64       
 4   total_spent       9977 non-null   float64       
 5   payment_method    6822 non-null   object        
 6   location          6039 non-null   object        
 7   transaction_date  9540 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 625.1+ KB


So, finally I have calculated respective values in all columns by imputing based on the information given

It is not possible to fill in information for payment_method and location as there are higher number of null values

### 7) Checking the possibility of imputing values for location

Location column shows there are significant null values

In [37]:
df[df['location'].isnull()]

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
3,TXN_7034554,Salad,2.0,5.0,10.0,NaN,NaN,2023-04-27
5,TXN_2602893,Smoothie,5.0,4.0,20.0,Credit Card,NaN,2023-03-31
7,TXN_6699534,Sandwich,4.0,4.0,16.0,Cash,NaN,2023-10-28
16,TXN_3765707,Sandwich,1.0,4.0,4.0,NaN,NaN,2023-06-10
18,TXN_8876618,Cake,5.0,3.0,15.0,Cash,NaN,2023-03-25
...,...,...,...,...,...,...,...,...
9994,TXN_7851634,Sandwich,4.0,4.0,16.0,NaN,NaN,2023-01-08
9995,TXN_7672686,Coffee,2.0,2.0,4.0,NaN,NaN,2023-08-30
9996,TXN_9659401,Cookie,3.0,1.0,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4.0,2.0,8.0,Digital Wallet,NaN,2023-03-02


In [38]:
df[df['item'] == 'Salad']

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
3,TXN_7034554,Salad,2.0,5.0,10.0,NaN,NaN,2023-04-27
10,TXN_2548360,Salad,5.0,5.0,25.0,Cash,Takeaway,2023-11-07
15,TXN_2847255,Salad,3.0,5.0,15.0,Credit Card,In-store,2023-11-15
37,TXN_1080432,Salad,2.0,5.0,10.0,Credit Card,In-store,2023-04-29
45,TXN_5455792,Salad,3.0,5.0,15.0,Cash,NaN,2023-03-22
...,...,...,...,...,...,...,...,...
9940,TXN_8273780,Salad,2.0,5.0,10.0,Digital Wallet,Takeaway,2023-10-15
9942,TXN_5344848,Salad,1.0,5.0,5.0,Digital Wallet,Takeaway,2023-09-27
9958,TXN_4125474,Salad,2.0,5.0,10.0,Credit Card,In-store,2023-08-02
9971,TXN_6120851,Salad,5.0,5.0,25.0,Cash,Takeaway,2023-02-04


In [39]:
df[df['item'] == 'Salad']['location'].value_counts().sum()

np.int64(784)

In [40]:
df[df['item'] == 'Salad']['location'].isna().value_counts(normalize= True, dropna= False)

location
False    0.616352
True     0.383648
Name: proportion, dtype: float64

In [41]:
df[df['item'] == 'Salad']['location'].value_counts(normalize= True, dropna= False)

location
NaN         0.383648
In-store    0.319182
Takeaway    0.297170
Name: proportion, dtype: float64

On evaluating above information it shows there are 40% of null values in the column.

Further, it shows orders from In-store and Take-away values are equally distributed. So, randomly allocating the type of order can apply bias to the analysis

It is better to leave it or categorise it as "other" for further analysis

In [42]:
df['location'] = df['location'].fillna('Other')

In [43]:
df[df['item'] == 'Salad']['location'].value_counts(normalize= True, dropna= False)

location
Other       0.383648
In-store    0.319182
Takeaway    0.297170
Name: proportion, dtype: float64

I have categorised them empty rows as 'Other' for further analysis

### 8) Checking possibiilty of imputing for 'payment_method'

Payment method column shows there are significant null values

In [44]:
df[df['item'] == 'Salad']

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
3,TXN_7034554,Salad,2.0,5.0,10.0,NaN,Other,2023-04-27
10,TXN_2548360,Salad,5.0,5.0,25.0,Cash,Takeaway,2023-11-07
15,TXN_2847255,Salad,3.0,5.0,15.0,Credit Card,In-store,2023-11-15
37,TXN_1080432,Salad,2.0,5.0,10.0,Credit Card,In-store,2023-04-29
45,TXN_5455792,Salad,3.0,5.0,15.0,Cash,Other,2023-03-22
...,...,...,...,...,...,...,...,...
9940,TXN_8273780,Salad,2.0,5.0,10.0,Digital Wallet,Takeaway,2023-10-15
9942,TXN_5344848,Salad,1.0,5.0,5.0,Digital Wallet,Takeaway,2023-09-27
9958,TXN_4125474,Salad,2.0,5.0,10.0,Credit Card,In-store,2023-08-02
9971,TXN_6120851,Salad,5.0,5.0,25.0,Cash,Takeaway,2023-02-04


In [45]:
df[df['item'] == 'Salad']['payment_method'].value_counts(normalize= True, dropna= False)

payment_method
NaN               0.295597
Cash              0.248428
Credit Card       0.240566
Digital Wallet    0.215409
Name: proportion, dtype: float64

On checking the distribution of null values it is found that almost 30% of values are missing.

Checking other categories show that they are almost equal in proportion expcept Digital Wallet payment.

Similar to 'Location' variable we can categorise as other method of payment for empty rows and analyse it further

In [46]:
df['payment_method'] = df['payment_method'].fillna('Other methods')

In [47]:
df[df['item'] == 'Salad']['payment_method'].value_counts(normalize= True, dropna= False)

payment_method
Other methods     0.295597
Cash              0.248428
Credit Card       0.240566
Digital Wallet    0.215409
Name: proportion, dtype: float64

As the number of rows with null values is very less compared to the total number of rows we can safely drop the null values

So, finally changed it to other methods of payment for empty rows for further analysis